In [1]:
import matplotlib.pyplot as plt 
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib
import time
from matplotlib.widgets import Slider, Button


In [2]:
matplotlib.use('tkAgg')



In [3]:
# %matplotlib widget
# %matplotlib inline


In [4]:
# #kOORD:       [  X   ,  Y  ,  Z  ]
# H1E=np.array([ 480.0, 6.33,-8.38]) 
# H1W=np.array([ 480.0,-6.64,-8.38]) 
# H2W=np.array([ 360.0,-6.64,-6.64]) 
# H3E=np.array([ 240.0, 6.33,-4.45])
# H3W=np.array([ 240.0,-6.64,-4.45]) 
# H4E=np.array([ 120.0, 6.33,-2.48]) 
# H4W=np.array([ 120.0,-6.64,-2.48]) 
# H5E=np.array([-7.0  , 6.33, -0.4]) 
# H5W=np.array([-7.0  ,-6.64, -0.4]) 
# H6E=np.array([-120.0, 6.33,-2.25]) 
# H6W=np.array([-120.0,-6.64,-2.25]) 
# H7E=np.array([-240.0, 6.33,-4.22]) 
# H7W=np.array([-240.0,-6.64,-4.22]) 
# H8E=np.array([-360.0, 6.33,-6.18]) 
# H9E=np.array([-480.0, 6.33,-8.15]) 
# H9W=np.array([-480.0,-6.64,-8.15])
# H2E_int= (H1E+H3E)/2
# H8W_int = (H7W+H9W)/2

# DOFs=np.array([H1E,H1W,H2E_int,H2W,H3E,H3W,H4E,H4W,H5E,H5W,H6E,H6W,H7E,H7W,H8E,H8W_int,H9E,H9W])
# DOFs=np.flip(DOFs,axis=0)

# # print(DOFs)
# def connectivity(DOFs,index):
#     x_out=[]
#     y_out=[]
#     z_out=[]        
#     for i in index:
#         for j,DOF in enumerate(DOFs):
#             x=[]
#             y=[]
#             z=[]            
#             x.append(DOFs[j,0])   # start points for each line
#             y.append(DOFs[j,1])   
#             z.append(DOFs[j,2]) 
            
#             if i+j>len(DOFs)-1:
#                 x.append(DOFs[j,0])   
#                 y.append(DOFs[j,1])   
#                 z.append(DOFs[j,2])   
#                 continue
#             if j//2 != j/2:

#                 if i==2: 
#                     x.append(DOFs[j+i,0])   
#                     y.append(DOFs[j+i,1])   
#                     z.append(DOFs[j+i,2])  
#                 else:
#                     x.append(DOFs[j,0])   
#                     y.append(DOFs[j,1])   
#                     z.append(DOFs[j,2]) 

#             else:
#                 x.append(DOFs[j+i,0])   
#                 y.append(DOFs[j+i,1])   
#                 z.append(DOFs[j+i,2])   
#             x_out.append(x)   
#             y_out.append(y)   
#             z_out.append(z)   
#     return np.array(x_out),np.array(y_out),np.array(z_out)

# x,y,z=connectivity(DOFs,[1,2])

# data=[]
# for i in range(len(x)):

#     data.append(x[i])
#     data.append(y[i])
#     # data.append(z[i])
# print(DOFs[1])

In [5]:
def deformed(phi,fac):
    '''Adding i deformation vector to the original coordinates of the sensors
    Arrguments: 
        phi: 1darray
            length 48
            [x1,y1,z1,x2....]
        fac: float
            Factor for scaling the deformations 
    returns: 
        DOFs: 1darray
            length 54 (!!!) 
            H2E and H8W are interpolated linearly from the neighbouring points.
    '''
    #kOORD:       [  X   ,  Y  ,  Z  ]
    H1E=np.array([ 480.0, 6.33,-8.38]) 
    H1W=np.array([ 480.0,-6.64,-8.38])
    H2W=np.array([ 360.0,-6.64,-6.64]) 
    H3E=np.array([ 240.0, 6.33,-4.45])
    H3W=np.array([ 240.0,-6.64,-4.45]) 
    H4E=np.array([ 120.0, 6.33,-2.48]) 
    H4W=np.array([ 120.0,-6.64,-2.48]) 
    H5E=np.array([-7.0  , 6.33, -0.4]) 
    H5W=np.array([-7.0  ,-6.64, -0.4]) 
    H6E=np.array([-120.0, 6.33,-2.25]) 
    H6W=np.array([-120.0,-6.64,-2.25]) 
    H7E=np.array([-240.0, 6.33,-4.22]) 
    H7W=np.array([-240.0,-6.64,-4.22]) 
    H8E=np.array([-360.0, 6.33,-6.18]) 
    H9E=np.array([-480.0, 6.33,-8.15]) 
    H9W=np.array([-480.0,-6.64,-8.15])

    DOFs_sensors=np.array([H1E,H1W,H2W,H3E,H3W,H4E,H4W,H5E,H5W,H6E,H6W,H7E,H7W,H8E,H9E,H9W])
    for i in range(len(DOFs_sensors)):
        DOFs_sensors[i,0]+=phi[3*i]*fac
        DOFs_sensors[i,1]+=phi[3*i+1]*fac
        DOFs_sensors[i,2]+=phi[3*i+2]*fac

    H2E_int= (DOFs_sensors[0]+DOFs_sensors[3])/2
    H8W_int = (DOFs_sensors[12]+DOFs_sensors[15])/2

    DOFs=np.insert(DOFs_sensors,2,H2E_int,axis=0)
    DOFs=np.insert(DOFs,15,H8W_int,axis=0)
    DOFs=np.flip(DOFs,axis=0)
    return DOFs




In [17]:
def create_phi(x,y,z):
    '''Create a phi vector based on x, y, z - vectors'''
    phi=[]
    for i in range(len(x)): 
        phi.append(x[i])
        phi.append(y[i])
        phi.append(z[i])
    return np.array(phi)


t=np.linspace(0,6,16)
z=np.sin(t)

phi=create_phi(z*0,z*0,z)
# print(phi)
DOFs=deformed(phi,100)

In [7]:
i=range(0,len(DOFs)-1,1)

def plottt(DOFs,ax,i):
    ax.scatter3D(DOFs[:,0],DOFs[:,1],DOFs[:,2])
    for j in i:
        if j<len(DOFs)-2:
            plot(ax,DOFs,j,j+2)
        if j%2 ==0:
           plot(ax,DOFs,j,j+1) 

def plot(ax,DOFs,i,j):
    ax.plot([DOFs[i,0],DOFs[j,0]],[DOFs[i,1],DOFs[j,1]],[DOFs[i,2],DOFs[j,2]],color='black')



In [19]:
def show_mode_3d(phi):
    matplotlib.use('tkAgg')
    

    fig = plt.figure(figsize=(10,6))
    ax=fig.add_subplot(111,projection='3d')
    ax.set_box_aspect((1, 0.05, 0.01))
    # ax.set_aspect(1.5)
    # def update()
    # ax.scatter3D(DOFs[:,0],DOFs[:,1],DOFs[:,2])
    ax.set_axis_off()
    i=range(0,len(DOFs)-1,1)
    ax.set_zlim(-20,10)


    axfreq = fig.add_axes([0.05, 0.05, 0.03, 0.6])
    fac_slider = Slider(
    ax=axfreq,
    label='Amplification\n factor',
    valmin=0,
    valmax=110,
    valinit=10,
    orientation="vertical"
    )

    axfreq = fig.add_axes([0.1, 0.05, 0.03, 0.6])
    t_slider = Slider(
    ax=axfreq,
    label='Phase',
    valmin=-1,
    valmax=1,
    valinit=1,
    orientation="vertical"
    )

    def update(val): 
        ax.clear()
        DOFs=deformed(phi,fac_slider.val*t_slider.val*10)
        ax.set_zlim(-20,10)
        ax.set_axis_off()
        plottt(DOFs,ax,i)
        fig.canvas.draw_idle()
    update(100)


    fac_slider.on_changed(update)
    t_slider.on_changed(update)
    plt.show()



show_mode_3d(phi)

In [ ]:
x = [1, 2, 3]
y = np.array([[1, 2], [3, 4], [5, 6]])
plt.plot(x, y)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button


# The parametrized function to be plotted
def f(t, amplitude, frequency):
    return amplitude * np.sin(2 * np.pi * frequency * t)

t = np.linspace(0, 1, 1000)

# Define initial parameters
init_amplitude = 5
init_frequency = 3

# Create the figure and the line that we will manipulate
fig, ax = plt.subplots()
line, = ax.plot(t, f(t, init_amplitude, init_frequency), lw=2)
ax.set_xlabel('Time [s]')

# adjust the main plot to make room for the sliders
fig.subplots_adjust(left=0.25, bottom=0.25)

# Make a horizontal slider to control the frequency.
axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
freq_slider = Slider(
    ax=axfreq,
    label='Frequency [Hz]',
    valmin=0.1,
    valmax=30,
    valinit=init_frequency,
)

# Make a vertically oriented slider to control the amplitude
axamp = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
amp_slider = Slider(
    ax=axamp,
    label="Amplitude",
    valmin=0,
    valmax=10,
    valinit=init_amplitude,
    orientation="vertical"
)


# The function to be called anytime a slider's value changes
def update(val):
    line.set_ydata(f(t, amp_slider.val, freq_slider.val))
    fig.canvas.draw_idle()


# register the update function with each slider
freq_slider.on_changed(update)
amp_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')


def reset(event):
    freq_slider.reset()
    amp_slider.reset()
button.on_clicked(reset)

plt.show()

KeyboardInterrupt: 